# The Analyzer

In [1]:
# Libraries
import json
import urllib
# import folium
import requests
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'folium'

In [ ]:
df = pd.read_csv('wine_data.csv', 
                 sep=',', 
                 encoding='utf-8')

In [ ]:
import requests
import urllib.parse

geolocation = []
n = 1
for i in df['land'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['land']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(i) +'?format=json'
        response = requests.get(url).json()
        geolocation.append(pd.Series(data={'lat': response[0]["lat"], 'lon': response[0]["lon"]}))

    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['land'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(50)

Geocoding address 250 out of 2799 : Frankreich
